<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Нужно построить модель со значением метрики качества *F1* не меньше 0.75. 


## Введение

Задача проекта - нахождение моделью токсичных комментариев. Для этого мы должны обучить модель, которая сможет классифицировать - негативную или позитивную каннотацию имеет текст комментария. Мы подготовим данные (загрузим их, ознакомимся с ними, проведём нормализацию текстов - токенизацию, лемматизацию, уберём стоп-слова и пунктуацию), обучим несколько моделей и выберем лучшую. Для проверки эффективности моделей мы будем использовать метрику f1. Выбирать модель будем исходя из значения метрики f1 на валидационной выборке. Затем лучшую модель проверим на тестовой выборке. 

## Подготовка

Произведём необходимые "импорты" и загрузим данные.

In [300]:
import pandas as pd
import re
from pymystem3 import Mystem
import spacy

from tqdm import tqdm
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 

In [108]:
df = pd.read_csv('C:/Users/p_kok/Downloads/toxic_comments.csv')
print(df.head())

   Unnamed: 0                                               text  toxic
0           0  Explanation\nWhy the edits made under my usern...      0
1           1  D'aww! He matches this background colour I'm s...      0
2           2  Hey man, I'm really not trying to edit war. It...      0
3           3  "\nMore\nI can't make any real suggestions on ...      0
4           4  You, sir, are my hero. Any chance you remember...      0


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Пропусков нет, типы данных корректные. Проверим, все ли данные размечены адекватно: посмотрим, какие значения есть в столбце toxic

In [110]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Всё в порядке, в столбце toxic только 0 и 1 (комментарий токсичен или нетоксичен). Переходим к подготовке данных. Первый этап - лемматизация и удаление стоп-слов.

In [112]:
#ВНИМАНИЕ!!! лемматизация в этой ячейке идёт около 2 часов
nlp = spacy.load("en_core_web_sm")
tqdm.pandas()
df['lemm'] = df['text'].progress_apply(lambda text: " ".join(token.lemma_ for token in nlp(text) if not token.is_stop)) 

100%|████████████████████████████████████████████████████████████████████████| 159292/159292 [2:05:00<00:00, 21.24it/s]


Посмотрим на результат:

In [228]:
df

,Unnamed: 0,text,toxic,lemm
0,0,Explanation\nWhy the edits made under my usern...,0,explanation \n edit username Hardcore Metallic...
1,1,D'aww! He matches this background colour I'm s...,0,d'aww ! match background colour seemingly stuc...
2,2,"Hey man, I'm really not trying to edit war. It...",0,"hey man , try edit war . guy constantly remove..."
3,3,"""\nMore\nI can't make any real suggestions on ...",0,""" \n \n real suggestion improvement - wonder s..."
4,4,"You, sir, are my hero. Any chance you remember...",0,", sir , hero . chance remember page ?"
...,...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0,""" : : : : : second time ask , view completely ..."
159288,159447,You should be ashamed of yourself \n\nThat is ...,0,ashamed \n\n horrible thing talk page . 128....
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0,"Spitzer \n\n Umm , s actual article prostituti..."
159290,159449,And it looks like it was actually you who put ...,0,look like actually speedy version delete look .


Следующий этап - удаление пунктуации.

In [246]:
%%time

#время радоты ячейки около 12 минут

for i in range(len(df['lemm'])):
    df['lemm'][i] = re.sub(r'[^\w\s]','', df['lemm'][i])



<timed exec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 9min 36s


In [247]:
df

,Unnamed: 0,text,toxic,lemm
0,0,Explanation\nWhy the edits made under my usern...,0,explanation \n edit username Hardcore Metallic...
1,1,D'aww! He matches this background colour I'm s...,0,daww match background colour seemingly stuck ...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man try edit war guy constantly remove r...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,\n \n real suggestion improvement wonder sec...
4,4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page
...,...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0,second time ask view completely contrad...
159288,159447,You should be ashamed of yourself \n\nThat is ...,0,ashamed \n\n horrible thing talk page 12861...
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer \n\n Umm s actual article prostitutio...
159290,159449,And it looks like it was actually you who put ...,0,look like actually speedy version delete look


<div class="alert alert-block alert-success">
<b>Успех:</b> Очистка и лемматизация были сделаны корректно.
</div>

## Обучение

Данные подготовлены, переходим к обучение моделей. Сначала разделим датасет на обучающую, валидационную и тестовую выборки. 

In [264]:
train, valid_test = train_test_split(df, train_size=0.81, random_state=12345, stratify=df['toxic'])
valid, test = train_test_split(valid_test, train_size=0.5, random_state=12345)

Проверим, корректно ли прошло разделение

In [265]:
print(test.shape, 'Test: {:.0%}'.format(len(test)/len(df)))
print(valid.shape, 'Valid: {:.0%}'.format(len(valid)/len(df)))
print(train.shape, 'Train: {:.0%}'.format(len(train)/len(df)))

(15133, 4) Test: 10%
(15133, 4) Valid: 10%
(129026, 4) Train: 81%


Рассчитаем tf-idf для всех выборок

In [267]:
vectorizer = TfidfVectorizer().fit(train['lemm'].values)
tf_idf_train = vectorizer.transform(train['lemm'].values)
tf_idf_valid = vectorizer.transform(valid['lemm'].values)
tf_idf_test = vectorizer.transform(test['lemm'].values)


Определим признаки и целевой признак для всех выборок

In [268]:
features_train = tf_idf_train
target_train = train['toxic']

features_valid = tf_idf_valid
target_valid = valid['toxic']

features_test = tf_idf_test
target_test = test['toxic']

Обучим модель дерева решений

In [269]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=7)
model_tree.fit(features_train, target_train) 
predicted = model_tree.predict(features_valid)
f1_tree = f1_score(target_valid, predicted)
print("f1 для решающего дерева:", f1_tree)

f1 для решающего дерева: 0.5653742110009017


Значение метрики f1 недостаточно высоко. Попробуем подобрать более подходящую глубину дерева.

In [270]:

for depth in range(1, 30):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predicted = model_tree.predict(features_valid)
    result = f1_score(target_valid, predicted)
    print('max_depth =', depth, ':', result)

max_depth = 1 : 0.3106904231625835
max_depth = 2 : 0.38165524512387977
max_depth = 3 : 0.43243243243243246
max_depth = 4 : 0.4742574257425743
max_depth = 5 : 0.502906976744186
max_depth = 6 : 0.542230517965469
max_depth = 7 : 0.5653742110009017
max_depth = 8 : 0.577636939791761
max_depth = 9 : 0.5972651080723422
max_depth = 10 : 0.6073943661971831
max_depth = 11 : 0.6135871916919082
max_depth = 12 : 0.6217034154777346
max_depth = 13 : 0.6264020707506471
max_depth = 14 : 0.6335190741534505
max_depth = 15 : 0.6308551783412119
max_depth = 16 : 0.6420787929589271
max_depth = 17 : 0.6484670306593868
max_depth = 18 : 0.6547717842323652
max_depth = 19 : 0.6595306710580485
max_depth = 20 : 0.6620123203285421
max_depth = 21 : 0.6729124236252545
max_depth = 22 : 0.6742671009771988
max_depth = 23 : 0.6804374240583233
max_depth = 24 : 0.68261045804621
max_depth = 25 : 0.6807473598700243
max_depth = 26 : 0.6879289463060153
max_depth = 27 : 0.690850463522773
max_depth = 28 : 0.6877022653721683
max_d

KeyboardInterrupt: 

Лучшая глубина - 27. Но такого значения метрики f1 недостаточно. Обучим модель случайного леса.

In [299]:
model_forest = RandomForestClassifier(max_depth=27, n_estimators=100, min_samples_leaf= 15, random_state=12345)
model_forest.fit(features_train, target_train) 
predicted = model_forest.predict(features_valid)
f1_forest = f1_score(target_valid, predicted)
print("RMSE для случайного леса:", f1_forest)

RMSE для случайного леса: 0.0013218770654329147


Обучим модель логистической регресии

In [302]:
model_lr = LogisticRegression(C=1.0, penalty='l2', random_state = 12345, max_iter = 70)
model_lr.fit(features_train, target_train)
prediction = model_lr.predict(features_valid)
f1 = f1_score(target_valid, prediction)
print(f1)

0.7498033044846579


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Значение метрики f1 близко к искомому, попробуем подобрать параметры, чтобы увеличить значение метрики

In [304]:
for par in range(1, 10):
    model_lr = LogisticRegression(C=par, penalty='l2', random_state = 12345, max_iter = 100)
    model_lr.fit(features_train, target_train)
    prediction = model_lr.predict(features_valid)
    f1 = f1_score(target_valid, prediction)
    print('C =', par, ':', f1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C = 1 : 0.7491138243402914


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C = 2 : 0.7809885931558935


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C = 7 : 0.7939661515820456


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C = 8 : 0.7916207276736494
C = 9 : 0.7921081476068688


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Логистическая регрессия показала хороший результат с параметром C = 6, его и выберем для проверки на тестовой выборке. 

In [310]:
model_lr = LogisticRegression(C=6.0, penalty='l2', random_state = 12345, max_iter = 100)
model_lr.fit(features_train, target_train)
prediction = model_lr.predict(features_valid)
f1 = f1_score(target_valid, prediction)
print(f1)

0.7940850277264324


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Проверим логистическую регрессию с выбранными параметрами на тестовой выборке. 

In [311]:
prediction = model_lr.predict(features_test)
f1 = f1_score(target_test, prediction)
print(f1)

0.7702312138728323


## Выводы

Для решения задачи классификации комментария как токсичного или нетоксичного мы подготовили данные (провели лемматизацию, избавились от стоп-слов и пунктуации) и обучили несколько моделей. Лучший результат показала модель логистической регрессии, которую мы и рекомендуем для решения данной задачи. 